##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Gemini进行笔记整理


## 准备工作

In [1]:
%pip install -U -q "google-genai>=1.16.0"
%pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.3 MB/s eta 0:00:00


In [3]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)
MODEL_ID="gemini-2.5-pro" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro", "gemini-2.5-flash-preview-05-20", "gemini-2.5-pro-preview-06-05"] {"allow-input":true, isTemplate: true}
import json
from PIL import Image
from IPython.display import display, Markdown
# 文件读取
from docx import Document
import os
import re


##  1. 从音频提取知识点，得到详细版笔记

In [11]:
import os
import re
from docx import Document

# ========== 基本路径设置 ==========
input_folder = r"/content/input"      # 你放原始 docx 文件的文件夹路径
output_folder = r"/content/results"        # 输出 result.txt 的文件夹

# 确保输出文件夹存在
os.makedirs(output_folder, exist_ok=True)

# 遍历文件夹中的所有 docx 文件
for filename in os.listdir(input_folder):
    if filename.endswith(".docx"):
        file_path = os.path.join(input_folder, filename)

        # 从文件名中提取 P 后面的数字
        match = re.search(r"P(\d+)", filename)
        if match:
            num = int(match.group(1)) - 1  # 提取数字并减 1
            name_part = f"S{num}"          # 生成 S16 这样的命名
        else:
            name_part = os.path.splitext(filename)[0]  # 如果没匹配到，就用原名

        # 读取文档内容
        doc = Document(file_path)
        text_content = "\n".join([para.text for para in doc.paragraphs])

        # 构造 prompt
        prompt = f"""
        请将以下课程音频的转写文本进行整理与提炼，提取出每个单词的知识点，输出为结构化的 Markdown 文件。

        格式要求如下：

        单词（Word）
        【释】[词性]. [含义1]；[词性]. [含义2]（如有多个含义，均列出；若为动词，请标明 vt. / vi.）
        【例】
        “[例句1]” ——（解释例句中该单词的含义）
        “[例句2]” ——（展示固定搭配或常用短语）
        【记】说明记忆方式（如词根词缀、联想、场景记忆等）
        【衍】列出主要衍生词，并说明词性和含义
        【同】列出常见同义词（如有细微语义区别，可补充简要辨析）
        【反】列出常见反义词

        每个单词之间请用三条连字符 --- 分隔，并在前后各保留一个空行。

        处理规则：

        仅根据音频内容提取知识点，不添加未提及信息；
        若某项内容未出现，请直接省略该行；
        所有输出必须格式对齐、整洁、无多余缩进；
        输出为纯 Markdown 文本，便于导入 Notion / Obsidian。
        {text_content}
        """
        thinking_budget = 4096 # @param {type:"slider", min:0, max:24576, step:1}

        response = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt,
            config=types.GenerateContentConfig(
                thinking_config=types.ThinkingConfig(
                    thinking_budget=thinking_budget
                )
            )
        )

        # 保存结果
        output_path = os.path.join(output_folder, f"{name_part}_result.txt")
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(response.text)

        print(f"已处理文件：{filename} → {output_path}")

print("\n全部处理完成 ✅")

已处理文件：P36_原文.docx → /content/results/S35_result.txt
已处理文件：P38_原文.docx → /content/results/S37_result.txt
已处理文件：P34_原文.docx → /content/results/S33_result.txt
已处理文件：P46_原文.docx → /content/results/S45_result.txt
已处理文件：P26_原文.docx → /content/results/S25_result.txt
已处理文件：P47_原文.docx → /content/results/S46_result.txt
已处理文件：P49_原文.docx → /content/results/S48_result.txt
已处理文件：P33_原文.docx → /content/results/S32_result.txt
已处理文件：P42_原文.docx → /content/results/S41_result.txt
已处理文件：P27_原文.docx → /content/results/S26_result.txt
已处理文件：P51_原文.docx → /content/results/S50_result.txt
已处理文件：P25_原文.docx → /content/results/S24_result.txt
已处理文件：P41_原文.docx → /content/results/S40_result.txt
已处理文件：P40_原文.docx → /content/results/S39_result.txt
已处理文件：P50_原文.docx → /content/results/S49_result.txt
已处理文件：P48_原文.docx → /content/results/S47_result.txt
已处理文件：P28_原文.docx → /content/results/S27_result.txt
已处理文件：P29_原文.docx → /content/results/S28_result.txt
已处理文件：P32_原文.docx → /content/results/S31_result.txt
已处理文件：P37_原文

In [12]:
import shutil
from google.colab import files

# 将文件夹打包为 zip 文件
shutil.make_archive('results', 'zip', '/content/results')

# 下载 zip 文件
files.download('results.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# GRE版

# 加载 Word 文档
doc = Document('/content/GRE单词地基词全集精讲 ｜ 北大学姐真人带背&串讲考点 ｜ 比GRE3000&佛脚词汇更高效 p07 S6-42个单词 - P07_原文.docx')

# 打开 txt 文件准备写入
with open('output.txt', 'w', encoding='utf-8') as txt_file:
    # 遍历文档中的所有段落
    for para in doc.paragraphs:
        txt_file.write(para.text + '\n')  # 将每个段落写入 txt 文件并换行


filename = r"output.txt"
with open(filename, 'r', encoding='utf-8') as f:
    text_content = f.read()

# 构造 prompt
prompt = f"""
请将以下课程音频的转写文本进行整理与提炼，提取出每个单词的知识点，输出为结构化的 Markdown 文件。

格式要求如下：

单词（Word）
【释】[词性]. [含义1]；[词性]. [含义2]（如有多个含义，均列出；若为动词，请标明 vt. / vi.）
【例】
“[例句1]” ——（解释例句中该单词的含义）
“[例句2]” ——（展示固定搭配或常用短语）
【记】说明记忆方式（如词根词缀、联想、场景记忆等）
【衍】列出主要衍生词，并说明词性和含义
【同】列出常见同义词（如有细微语义区别，可补充简要辨析）
【反】列出常见反义词

每个单词之间请用三条连字符 --- 分隔，并在前后各保留一个空行。

处理规则：

仅根据音频内容提取知识点，不添加未提及信息；

若某项内容未出现，请直接省略该行；

所有输出必须格式对齐、整洁、无多余缩进；

输出为纯 Markdown 文本，便于导入 Notion / Obsidian。
{text_content}
"""
thinking_budget = 4096 # @param {type:"slider", min:0, max:24576, step:1}

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_budget=thinking_budget
        )
    )
)

# 将 response.text 保存为 result.txt 文件
with open("result.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

print("保存完成：result.txt")

PackageNotFoundError: Package not found at '/content/GRE单词地基词全集精讲 ｜ 北大学姐真人带背&串讲考点 ｜ 比GRE3000&佛脚词汇更高效 p07 S6-42个单词 - P07_原文.docx'

In [ ]:
# TOFEL版

# 加载 Word 文档
doc = Document('/content/GRE单词地基词全集精讲 ｜ 北大学姐真人带背&串讲考点 ｜ 比GRE3000&佛脚词汇更高效 p01 S1-57个单词 - P01_原文.docx')

# 打开 txt 文件准备写入
with open('output.txt', 'w', encoding='utf-8') as txt_file:
    # 遍历文档中的所有段落
    for para in doc.paragraphs:
        txt_file.write(para.text + '\n')  # 将每个段落写入 txt 文件并换行


filename = r"output.txt"
with open(filename, 'r', encoding='utf-8') as f:
    text_content = f.read()

# 构造 prompt
prompt = f"""
请将以下的课程音频转写文件进行处理，提取知识点，整理成结构化的Markdown文件。格式要求如下：
------

# 单词（Word）

- **meanings**：
  - [词性]. [含义1]
  - [词性]. [含义2]（如有多个意思，均列出）
- **usage**：
  - "[例句1]"（本单词在该例句中的含义）
  - "[例句2]"（固定搭配或用法示例）
     （每条分行，条理清晰）
- **related words**：
  - **SYN**（近义词）：
    - [近义词1]（词性. 含义）
      - [例句或补充说明]（如果用法超过两条，另起二级标题详细说明）
    - [近义词2]（词性. 含义）
      - [例句或补充说明]
  - **ANTONYM**（反义词）：
    - [反义词1]（词性. 含义）
  - **DERIVATIVE WORDS**（衍生词）：
    - [衍生词1]（词性. 含义）
    - [衍生词2]（词性. 含义）
      - [例句或补充说明]
- **mnemonic**：
  - 词根`[词根]` = [词根含义]，解释词义来源或联想
  - 其他记忆方法或助记技巧
- **notes**：
  - [补充说明、用法注意或其他无法归类内容]

------

**补充说明：**

- 若某一属性（meanings、usage、related words、mnemonic、notes）没有内容，则该属性可省略，不必空列。

- 若related word中某词的usage超过两条，需在该词下另用二级标题写出，且保持相同结构（meanings、usage、related words、mnemonic、notes）。
- 所有用例、短语等均需分行列出，确保阅读方便。
- 若某知识点不能归入 meanings、usage、related words、mnemonic 四类，则归入 notes。

{text_content}
"""
thinking_budget = 4096 # @param {type:"slider", min:0, max:24576, step:1}

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_budget=thinking_budget
        )
    )
)

# 将 response.text 保存为 result.txt 文件
with open("result.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

print("保存完成：result.txt")

保存完成：result.txt


## 2.详细版笔记精炼化
### 2.1 传入当天单词总表，分批次输入

In [ ]:
import re
import os
import shutil
def split_md_by_sr(filename, output_dir,split_pattern=r'(SR\d+)'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(filename, 'r', encoding='utf-8') as f:
        content = f.read()

    # 匹配 SR后面跟数字，捕获分割点
    pattern = re.compile(split_pattern)
    parts = pattern.split(content)
    for i, part in enumerate(parts):
      print(f"parts[{i}]: {repr(part)}")

    # parts格式： ['', 'SR1', 内容1, 'SR2', 内容2, ...]
    for i in range(1, len(parts), 2):
        title = parts[i].strip()  # SR编号
        body = parts[i+1].strip() if (i+1) < len(parts) else ''

        # 生成文件名，比如 SR1.txt
        out_path = os.path.join(output_dir, f"{title}.txt")

        with open(out_path, 'w', encoding='utf-8') as fout:
            # 保留编号作为第一行
            fout.write(f"{title}\n{body}")

    print(f"拆分完成，文件保存在：{output_dir}")

input_file = r"/content/SR11.txt"
output_folder = r"/content/files"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"文件夹已创建：{output_folder}")
else:
    print(f"文件夹已存在：{output_folder}")

# 删除文件夹里所有内容（文件和子文件夹）
for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)  # 删除文件或符号链接
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)  # 递归删除文件夹及里面所有内容
split_md_by_sr(input_file, output_folder, split_pattern=r'（SR\d+）')

### 2.2 汇总得到简化版笔记

In [ ]:
import os
import re
input_dir = "/content/files"
output_file = "/content/result.txt"

# 先清空result.md（如果已存在）
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("")
# 重新排序
def extract_number(filename):
    # 用正则匹配文件名中的数字部分
    match = re.search(r'(\d+)', filename)
    if match:
        return int(match.group(1))
    else:
        # 如果没有数字，返回一个大数，保证排序靠后
        return float('inf')
files = [f for f in os.listdir(input_dir) if f.endswith('.txt')]
files_sorted = sorted(files, key=extract_number)

thinking_budget = 4096 # @param {type:"slider", min:0, max:24576, step:1}
for filename in files_sorted:
    if not filename.endswith(".txt"):
        continue
    input_path = os.path.join(input_dir, filename)

    with open(input_path, 'r', encoding='utf-8') as f:
        text_content = f.read()
    prompt = f"""
    请你提取以下内容，并用 Markdown 格式输出：

    核心单词（用三级标题 ### 单词 标明）
    本单词的固定搭配

    本单词的用法

    本单词的例句

    衍生词

    衍生词的用法和例句

    要求：

    核心单词用三级标题标明，如 ### abide，每个单词只出现一次，位于该单词所有表达之前；

    其他表达以有序列表形式呈现，格式如下：

    markdown
    复制
    编辑
    1. "英文表达" (知识点单词 本句中涉及的意思或用法 中文翻译)
        * 例句（**加粗中文翻译，方便回想**）
    只提取英文表达，

    “知识点单词”、“本句中涉及的意思或用法”、“中文翻译”由你基于原文内容准确理解并用简洁中文表达，避免逐字照搬；

    例句紧跟在对应表达下面，用缩进的 * 标记单独一行展示，保持例句的英文原文和标点不变；

    删除所有笔记中的注释、解释、翻译等无关内容，只保留上述表达和例句；

    同一表达有多条例句时，每条例句均单独列出；

    输出顺序严格为：核心单词标题 → 固定搭配 → 用法 → 例句 → 衍生词 → 衍生词用法及例句；

    确保中文释义精准简洁，便于记忆与回想。

    示例输入（示范）：
    pgsql
    复制
    编辑
    # abound

    - Meaning: Exist in large numbers
      - Used as an intransitive verb to indicate abundance.
      - Example: "Stories about his travels abound" (关于他的游历的故事有很多).
    - Phrase: abound with
      - Means to be full of or rich in something.
      - Example: "The lake abounds with fish" (这个湖有很多鱼).
    示例输出：
    markdown
    复制
    编辑
    ### abound
    1. "abound" (动词 表示数量多)
        * Stories about his travels abound (关于他的游历的故事有很多).
    2. "abound with" (固定搭配 充满)
        * The lake abounds with fish (这个湖有很多鱼).

    {text_content}
    """
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(
                thinking_budget=thinking_budget
            )
        )
    )
  # 追加写入统一result.md文件，且用一级标题分割
    with open(output_file, 'a', encoding='utf-8') as fout:
        # 以文件名（去除扩展名）作为一级标题
        section_title = os.path.splitext(filename)[0]
        fout.write(f"# {section_title}\n\n")
        fout.write(response.text)
        fout.write("\n\n")
    print(f"{filename} 处理完成，结果追加写入 {output_file}")

print("保存完成：result.md")

SR11.txt 处理完成，结果追加写入 /content/result.md
SR12.txt 处理完成，结果追加写入 /content/result.md
SR13.txt 处理完成，结果追加写入 /content/result.md
SR14.txt 处理完成，结果追加写入 /content/result.md
SR15.txt 处理完成，结果追加写入 /content/result.md
SR17.txt 处理完成，结果追加写入 /content/result.md
SR18.txt 处理完成，结果追加写入 /content/result.md
SR19.txt 处理完成，结果追加写入 /content/result.md
SR20.txt 处理完成，结果追加写入 /content/result.md
保存完成：result.md


In [ ]:
# 文件读取
filename = r"/content/Art艺术.txt"
with open(filename, 'r', encoding='utf-8') as f:
    text_content = f.read()

# 构造 prompt
prompt = f"""
任务目标
将原始英文单词列表（包含可能的音标、词性、中文释义）转换成标准 Markdown 单词表，每个词条一行，列出英文单词 / 音标 / 中文释义三部分，同时保留章节标题或类别标记。
处理规则
1. 词条结构
    格式：` 单词  音标  中文释义 `
2. 单词识别
    单词通常位于音标之前，去除多余空格与换行。
    复合词或短语保持原格式（如 still life）。
    OCR 错误明显时尝试修正（如 countrvside → countryside）。
3. 音标提取
    识别方括号 `[]` 或斜杠 `/` 中的内容作为音标。
    去掉混入的词性标记（如 `n`, `adj`）和多余空格。
    若音标缺失，请给出。
4. 中文释义
    音标之后的中文内容作为释义。
    保留多义项（用 `；` 分隔），删除多余空格和标点。
    OCR 明显错误修正（如 `泊画`→`油画`，`真止的`→`真正的`，`观煮视角`→`观察视角`）。
5. 特殊处理
    同义词（`=`）或替代表达（`/`）分成多条独立词条。
    删除非单词的乱码、重复标记。
    保留章节标题和类别标记，可用 单独的一行：` Art艺术   ` 形式表示（音标和释义可留空）。
6. 输出要求
    按原文顺序输出词条，不做字母排序。
    章节标题和类别标记应作为独立行保留。
-
{text_content}
"""
thinking_budget = 4096 # @param {type:"slider", min:0, max:24576, step:1}

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_budget=thinking_budget
        )
    )
)

# 将 response.text 保存为 result.txt 文件
with open("result.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

print("保存完成：result.txt")

保存完成：result.txt
